In [3]:
import os
from glob import glob

import umap
from sklearn.manifold import TSNE

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist, pdist, squareform

import torch

from local_utils import (
    init_embeddings,
    calculate_concept_centers,
    calculate_concept_center_distances,
    compute_top_n_per_doppelganger_pair,
    compute_top_n_per_sample,
)

/home/nthom/miniconda3/envs/opensphere/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embeddings_path = "/home/nmichelotti/Desktop/Embeddings/embeddings_for_n8/model_240000_DoppelVer_All_112x112_outputs.pth"
embeddings_image_path = "/home/nmichelotti/Desktop/Embeddings/embeddings_for_n8/model_240000_DoppelVer_All_112x112_image_paths.txt"
image_dir = "/home/nmichelotti/Desktop/Embeddings/embeddings_for_n8/DoppelVer_All_112x112"
doppelganger_pairs_path = (
    "/home/nmichelotti/Desktop/Embeddings/embeddings_for_n8/doppelgangerPairs.csv"
)
embeddings_df = init_embeddings(embeddings_path, embeddings_image_path, image_dir)
print(len(embeddings_df["class"].unique()))

390


In [5]:
doppelganger_pairs_df = pd.read_csv(doppelganger_pairs_path)
doppelganger_pairs_df["Pair 1"] = doppelganger_pairs_df["Pair 1"].str.replace(" ", "_")
doppelganger_pairs_df["Pair 2"] = doppelganger_pairs_df["Pair 2"].str.replace(" ", "_")

In [6]:
concept_centers_df = calculate_concept_centers(embeddings_df)
concept_distance_df = calculate_concept_center_distances(concept_centers_df)
(
    doppelganger_pair_top_n_analysis_df,
    doppelganger_pair_acc_list,
) = compute_top_n_per_doppelganger_pair(
    concept_distance_df, doppelganger_pairs_df, [1, 2, 3, 5, 10]
)
(
    sample_top_n_analysis_df,
    # per_id_sample_top_n_analysis_df,
    sample_acc_list,
) = compute_top_n_per_sample(concept_centers_df, embeddings_df, [1, 2, 3, 5, 10])

In [7]:
print(doppelganger_pair_top_n_analysis_df)
print(doppelganger_pair_acc_list)


                Source ID   Doppelganger ID       Closest ID  Top 1 Acc.  \
0                 Beyonce           Solange    Anne_Hathaway       False   
1              Katy_Perry   Zooey_Deschanel    Wendie_Malick       False   
2             Hilary_Duff       Haylie_Duff     Henry_Cavill       False   
3             Ben_Affleck     Casey_Affleck     Sarah_Hyland       False   
4           Penelope_Cruz       Monica_Cruz     Henry_Cavill       False   
..                    ...               ...              ...         ...   
232  James_Patrick_Stuart    Christian_Bale      Rooney_Mara       False   
233           Stana_Katic  Elizabeth_Reaser      Nina_Dobrev       False   
234          Serinda_Swan    Kirsten_Storms      Hilary_Duff       False   
235             Brad_Pitt        Guy_Pearce      Hilary_Duff       False   
236          James_Franco       Dave_Franco  Keira_Knightley       False   

     Top 2 Acc.  Top 3 Acc.  Top 5 Acc.  Top 10 Acc.  
0         False       False     

In [8]:
print(sample_top_n_analysis_df)
print(sample_acc_list)

                Source ID            Closest ID  Top 1 Acc.  Top 2 Acc.  \
0           Zooey_Kravitz         Corey_Feldman       False       False   
1             Olivia_Munn      Guillermo_Zapata       False       False   
2            Hugh_Jackman         Corey_Feldman       False       False   
3           Raven_Goodwin  Christopher_Williams       False       False   
4              Katy_Perry         Corey_Feldman       False       False   
...                   ...                   ...         ...         ...   
33744     Guillaume_Canet           Jaden_Smith       False       False   
33745  Christy_Turlington           Amber_Heard       False       False   
33746     Luke_Youngblood         Tina_Majorino       False       False   
33747            Weird_Al      Daniel_Day-Lewis       False       False   
33748     America_Ferrera        Michael_Weston       False        True   

       Top 3 Acc.  Top 5 Acc.  Top 10 Acc.  
0           False       False        False  
1        